In [1]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'password'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Loyalty Program Management

## Description of the use case
This use case involves managing a loyalty program where customers earn points for their purchases on Credit Cards, which can be redeemed for discounts or rewards. This helps in retaining customers and encouraging repeat purchases.

## Redis Data Structures Used
- **Hashes**: To store customer loyalty points and reward information.
- **Sorted Sets**: To rank customers based on their loyalty points.

In [2]:
# Mock data for the use case
mock_data = {
    'cust_1': {'name': 'Amit', 'loyalty_points': 1200},
    'cust_2': {'name': 'Vikas', 'loyalty_points': 800},
    'cust_3': {'name': 'Akash', 'loyalty_points': 1500},
}

# Use Hashes to store customer loyalty points and reward information
for customer_id, customer_info in mock_data.items():
    r.hset(customer_id, mapping=customer_info)

# Use Sorted Sets to rank customers based on their loyalty points
for customer_id, customer_info in mock_data.items():
    r.zadd('customer_loyalty', {customer_id: customer_info['loyalty_points']})

# Function to add loyalty points to a customer
def add_loyalty_points(customer_id, points):
    r.hincrby(customer_id, 'loyalty_points', points)
    new_points = int(r.hget(customer_id, 'loyalty_points'))
    r.zadd('customer_loyalty', {customer_id: new_points})

# Add loyalty points to customers
add_loyalty_points('cust_1', 100)
add_loyalty_points('cust_2', 200)

# Print the updated customer loyalty information
for customer_id in mock_data.keys():
    print(f"{customer_id}: {r.hgetall(customer_id)}")

# Print the ranked customers by loyalty points
ranked_customers = r.zrange('customer_loyalty', 0, -1, withscores=True, desc=True)
print('Ranked Customers by Loyalty Points:')
for customer in ranked_customers:
    print(customer)

cust_1: {'name': 'Amit', 'loyalty_points': '1300'}
cust_2: {'name': 'Vikas', 'loyalty_points': '1000'}
cust_3: {'name': 'Akash', 'loyalty_points': '1500'}
Ranked Customers by Loyalty Points:
('cust_3', 1500.0)
('cust_1', 1300.0)
('cust_2', 1000.0)
